In [1]:
import torch 
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os

In [14]:
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'split_dataset')

train_dir = os.path.join(data_dir, 'train')

#test_realworld_dir  = os.path.join(data_dir, 'test_realworld')
#test_studio_dir  = os.path.join(data_dir, 'test_studio')

In [15]:
# image preprocessing: resize and convert to tensor 
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

In [16]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)